<a href="https://colab.research.google.com/github/KiyongAhn/rep01/blob/master/33_AutoEncoder101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoEncoder 101

### Your name :

#### 딥러닝의 세상에 한 발자국 더!

![AE](https://www.curiousily.com/media/data-imputation-2/mushroom_encoder.png)
--------------------------
Rayleigh Kim @ D:plus

### 빌려온 전처리용 파일들

[LFW](http://vis-www.cs.umass.edu/lfw/)라는 데이터 셋으로 연습을 할 예정!

Coursera의 [Introduction to Deep Learning](https://www.coursera.org/learn/intro-to-deep-learning) 강의에서 사용한 전처리 도구들을 빌려 사용할 예정!

**직접 전처리 하다가는 하루도 모자람.**

In [0]:
!git clone https://github.com/hse-aml/intro-to-dl.git

In [0]:
cd intro-to-dl

In [0]:
!pip install tqdm
!pip install ipywidgets

import tqdm
import ipywidgets
from IPython.display import clear_output
clear_output()

In [0]:
from download_utils import sequential_downloader as sd

sd(    "v0.4",
    [
        "lfw-deepfunneled.tgz",
        "lfw.tgz",
        "lfw_attributes.txt"
    ],
    "./week4")

In [0]:
cd week4

## 라이브러리 로딩

In [0]:
import sys
sys.path.append("..")

In [0]:
import tensorflow as tf

import keras, keras.layers as L, keras.backend as K

import numpy as np
from sklearn.model_selection import train_test_split
from lfw_dataset import load_lfw_dataset
%matplotlib inline
import matplotlib.pyplot as plt
import download_utils
import keras_utils
import numpy as np

import random

## Data Loading

In [0]:
X, attr = load_lfw_dataset(use_raw=True, dimx=32, dimy=32)
IMG_SHAPE = X.shape[1:]

# Scaling
X = X.astype('float32') / 255

# Split
train_x, test_x = train_test_split(X, test_size = 0.1, random_state=2018)

# try to free memory
del X
import gc
gc.collect()

In [0]:
## 반복 실행해보면, 어떤 얼굴들이 있는지 구경할 수 있다!

random_ids = random.sample(range(0,11828), 6)

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(train_x[  random_ids[i] ])
    
print('train_x shape : ',train_x.shape )
print('attr shape : ',attr.shape)

In [0]:
attr.head()

## First, Simple Auto Encoder

In [0]:
print(IMG_SHAPE)

In [0]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()


# Encoder!

encoder = keras.models.Sequential()
encoder.add(L.InputLayer(IMG_SHAPE))
encoder.add(L.Flatten())   ## 40*40*3개
encoder.add(L.Dense( 64 )) ## 64개의 숫자로 데이터를 축소! 인코딩!

# Decoder!

decoder = keras.models.Sequential()
decoder.add(L.InputLayer((64,  )  ))  ## 64개의 숫자로 인코딩된 벡터로 부터.
decoder.add(L.Dense( np.prod(IMG_SHAPE)     ))  ## Decoder!, 48*48*4개 노드 복원
decoder.add(L.Reshape(IMG_SHAPE)) ## Flatten 해제. 다시 이미지로.


# 인코더와 디코더 결합
inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inputs = inp, outputs=reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

# 요약
autoencoder.summary()

In [0]:
# 학습!

autoencoder.fit(x = train_x, y = train_x, epochs = 20,
               validation_split = 0.1, verbose = 1)

In [0]:
def show_image(x):
    plt.imshow(np.clip(x, 0, 1))

def visualize(img,encoder,decoder):
    """Draws original, encoded and decoded images"""
    code = encoder.predict(img[None])[0]  # img[None] is the same as img[np.newaxis, :]
    reco = decoder.predict(code[None])[0]

    plt.subplot(1,3,1)
    plt.title("Original")
    show_image(img)

    plt.subplot(1,3,2)
    plt.title("Code")
    plt.imshow(code.reshape([code.shape[-1]//2,-1]))

    plt.subplot(1,3,3)
    plt.title("Reconstructed")
    show_image(reco)
    plt.show()


In [0]:
score = autoencoder.evaluate(test_x, test_x, verbose = 1)
print('MSE : ', score)


In [0]:
random_ids = random.sample(range(1315), 5)
for i in range(5) :
    img = test_x[random_ids[i]]
    visualize(img,encoder, decoder)

## Second, Convolutional AutoEncoder

In [0]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()


H,W,C = IMG_SHAPE

# Encoder!

encoder = keras.models.Sequential()
encoder.add(L.InputLayer(IMG_SHAPE))

encoder.add(L.Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='elu'))
encoder.add(L.MaxPooling2D(pool_size=(2, 2)))

encoder.add(L.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='elu'))
encoder.add(L.MaxPooling2D(pool_size=(2, 2)))

encoder.add(L.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='elu'))
encoder.add(L.MaxPooling2D(pool_size=(2, 2)))

encoder.add(L.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='elu'))
encoder.add(L.MaxPooling2D(pool_size=(2, 2)))


encoder.add(L.Flatten())
encoder.add(L.Dense( 64 , activation = 'elu')) ## 64개의 숫자로 데이터를 축소! 인코딩!

# Decoder!

decoder = keras.models.Sequential()
decoder.add(L.InputLayer((64,  )  ))  ## 64개의 숫자로 인코딩된 벡터로 부터.


## 풀링 4번, -> 가로 세로 각각 2^4 만큼 축소

# 128개의 벡터로 부터 -> L.Flatten() 레이어로 복원
decoder.add(L.Dense(256 * H // (2 ** 4) * W // (2 ** 4))) 

# Flatten 해제
decoder.add(L.Reshape((H // (2 ** 4), W // (2 ** 4), 256)))

# Convolution 해제
decoder.add(L.Conv2DTranspose(filters=128, kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
decoder.add(L.Conv2DTranspose(filters=64,  kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
decoder.add(L.Conv2DTranspose(filters=32,  kernel_size=(3, 3), strides=2, activation='elu', padding='same'))
decoder.add(L.Conv2DTranspose(filters=3,   kernel_size=(5, 5), strides=2, activation=None,  padding='same'))
    




# 인코더와 디코더 결합
inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inputs = inp, outputs=reconstruction)
autoencoder.compile(optimizer='adamax' , loss='mse')

# 요약
autoencoder.summary()
encoder.summary()
decoder.summary()

In [0]:
# 학습!

autoencoder.fit(x = train_x, y = train_x, epochs = 40, batch_size= 2048,
               validation_split = 0.1, verbose = 1)

In [0]:
score = autoencoder.evaluate(test_x, test_x, verbose = 1)
print('MSE : ', score)

In [0]:
random_ids = random.sample(range(1315), 5)
for i in range(5) :
    img = test_x[random_ids[i]]
    visualize(img,encoder, decoder)

## Denoising Autoencoder ( Your Turn! )

In [0]:
def apply_gaussian_noise(X,sigma=0.1):
    noise = np.random.normal(loc=0.0, scale = sigma, size = X.shape)
    return X + noise

In [0]:
# test different noise scales
plt.subplot(1,4,1)
show_image(train_x[0])
plt.subplot(1,4,2)
show_image(apply_gaussian_noise(train_x[:1],sigma=0.01)[0])
plt.subplot(1,4,3)
show_image(apply_gaussian_noise(train_x[:1],sigma=0.1)[0])
plt.subplot(1,4,4)
show_image(apply_gaussian_noise(train_x[:1],sigma=0.5)[0])

In [0]:
train_x_noise = apply_gaussian_noise(train_x)
test_x_noise = apply_gaussian_noise(test_x)

In [0]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()


H,W,C = IMG_SHAPE

# Encoder!

encoder = keras.models.Sequential()
encoder.add(L.InputLayer(IMG_SHAPE))







encoder.add(L.Flatten())
encoder.add(L.Dense( 512 , activation = 'elu')) ## 512개의 숫자로 데이터를 축소! 인코딩!

# Decoder!

decoder = keras.models.Sequential()
decoder.add(L.InputLayer((512,  )  ))  ## 512개의 숫자로 인코딩된 벡터로 부터.


## 풀링 4번, -> 가로 세로 각각 2^4 만큼 축소

# 128개의 벡터로 부터 -> L.Flatten() 레이어로 복원

n =  # 풀링 횟수
decoder.add(L.Dense(256 * H // (2 ** n) * W // (2 ** n))) 

# Flatten 해제
decoder.add(L.Reshape((H // (2 ** n), W // (2 ** n), 256)))

# Convolution 해제



# 인코더와 디코더 결합
inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inputs = inp, outputs=reconstruction)
autoencoder.compile(optimizer='adamax' , loss='mse')

# 요약
autoencoder.summary()
encoder.summary()
decoder.summary()

In [0]:
# 학습!

autoencoder.fit(x = train_x_noise, y = train_x, epochs = 40, batch_size= 2048,
               validation_split = 0.1, verbose = 1)

In [0]:
score = autoencoder.evaluate(test_x_noise, test_x, verbose = 1)
print('MSE : ', score)

In [0]:
random_ids = random.sample(range(1315), 5)
for i in range(5) :
    img = test_x[random_ids[i]]
    visualize(img,encoder, decoder)

## Image retrieval with autoencoders

In [0]:
# 혹시 이미 그려둔 그래프가 있다면 날려줘!
keras.backend.clear_session()


# Encoder!

encoder = keras.models.Sequential()
encoder.add(L.InputLayer(IMG_SHAPE))
encoder.add(L.Flatten())   ## 40*40*3개
encoder.add(L.Dense( 64 )) ## 64개의 숫자로 데이터를 축소! 인코딩!

# Decoder!

decoder = keras.models.Sequential()
decoder.add(L.InputLayer((64,  )  ))  ## 64개의 숫자로 인코딩된 벡터로 부터.
decoder.add(L.Dense( np.prod(IMG_SHAPE)     ))  ## Decoder!, 48*48*4개 노드 복원
decoder.add(L.Reshape(IMG_SHAPE)) ## Flatten 해제. 다시 이미지로.


# 인코더와 디코더 결합
inp = L.Input(IMG_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)

autoencoder = keras.models.Model(inputs = inp, outputs=reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse')

# 요약
autoencoder.summary()

In [0]:
# 학습!

autoencoder.fit(x = train_x, y = train_x, epochs = 20,
               validation_split = 0.1, verbose = 1)

In [0]:
#### 인코딩된 벡터!
images = train_x
codes = encoder.predict(train_x)


In [0]:
from sklearn.neighbors.unsupervised import NearestNeighbors
nei_clf = NearestNeighbors(metric="euclidean")
nei_clf.fit(codes)

In [0]:
def get_similar(image, n_neighbors=5):
    assert image.ndim==3,"image must be [batch,height,width,3]"

    code = encoder.predict(image[None])
    
    (distances,),(idx,) = nei_clf.kneighbors(code,n_neighbors=n_neighbors)
    
    return distances,images[idx]

In [0]:
def show_similar(image):
    
    distances,neighbors = get_similar(image,n_neighbors=3)
    
    plt.figure(figsize=[8,7])
    plt.subplot(1,4,1)
    show_image(image)
    plt.title("Original image")
    
    for i in range(3):
        plt.subplot(1,4,i+2)
        show_image(neighbors[i])
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

Exercise 1.

아래 셀을 반복 실행 해가며,  

정말 비슷한 이미지들이 나올 경우 picture ID를 기록하고,  새로운 셀에 시각화 하여 공유하자.

* 5개의 ID를 찾아 공유 할 것.
* 비슷하다고 느껴지는 이유를 말해볼 것.

In [0]:
#######################

random_id = random.randrange(0,test_x.shape[0])

print('Picture ID : ',random_id)
show_similar(test_x[random_id])

In [0]:
###### 예시.
# 얼굴 각도가 다 똑같음

print('Picture ID : ',1154)
show_similar(test_x[1154])